In [105]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt  # Biblioteca para gerar gráficos
import pandas as pd
from sklearn import metrics, model_selection
from scipy import stats
import math


In [106]:
data = np.genfromtxt('breastcancer.csv', delimiter=',')

## Questão 1

Considere o conjunto de dados disponível em `breastcancer.csv`, organizado
em 31 colunas, sendo as 30 primeiras colunas os atributos e a última coluna a
saída. Os 30 atributos coletados de exames médicos são usados no diagnóstico
do câncer de mama, sendo 1 a classe positiva e 0 a classe negativa. Maiores 
detalhes sobre os dados podem ser conferidos em 
https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset.

a) Considerando uma validação cruzada em 10 folds, avalie modelos de classicação binária nos dados em questão. Para tanto, use as abordagens abaixo:
- Regressão logística (treinado com GD ou SGD);
- Análise do discriminante Gaussiano;
- Naive Bayes Gaussiano;

In [107]:

class Set:
    def __init__(self, dataset, features, output):
        self.dataset = dataset
        self.features = features
        self.output = output

    def get_n(self):
        return self.dataset.shape[0]

    def get_x(self):
        return self.dataset[:, self.features]

    def set_x(self, new_x):
        self.x = new_x

    def get_y(self):
        return self.dataset[:, self.output]

    def get_X(self, normal_fun=None):
        if (normal_fun):
            return np.c_[np.ones(self.get_n()), normal_fun(self.get_x())]
        else:
            return np.c_[np.ones(self.get_n()), self.get_x()]


In [108]:
def k_fold_split(array, k = int):
    shuffled_data = np.random.permutation(array)
    folds = np.array_split(shuffled_data, k)
    return folds

folds = k_fold_split(data, 10)

In [109]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def GD(X, y, learning_rate, epochs):
    n = y.shape[0]
    # w = np.random.randn(X.shape[1])
    w = np.zeros(X.shape[1])

    for t in range(epochs):
        ei_sum = np.zeros(X.shape[1])
        for i in range(n):
            ei = y[i] - sigmoid(np.dot(w.T, X[i]))
            ei_sum += np.dot(ei, X[i])
        w = w + learning_rate * (ei_sum/n)
        # w = w + learning_rate * np.mean(np.dot(e, X))
    return w

def log_reg_GD(train_set, test_set, learning_rate, epochs):
    features = np.arange(30)
    output = 30

    train_set = Set(train_set, features, output)
    X_train = train_set.get_X(normal_fun=stats.zscore)
    y_train = train_set.get_y()

    w_train = GD(X_train, y_train, learning_rate, epochs)

    test_set = Set(test_set, features, output)
    X_test = test_set.get_X(normal_fun=stats.zscore)
    y_test = test_set.get_y()

    y_pred = sigmoid(np.dot(w_train, X_test.T))
    y_pred = np.around(y_pred)

    return (y_test, y_pred)


In [110]:
# todo analise de discriminante gaussiano
def GDA():
    pass

In [111]:
# todo naive bayes
def naibe_bayes():
    pass

b) Para cada modelo criado, reporte valor médio e desvio padrão das métricas de **acurácia**, **revocação**, **precisão** e **F1-score**

In [112]:
def is_true_positive(y, y_pred):
    return y_pred >= 1 and y >= 1

def is_false_positive(y, y_pred):
    return y_pred >= 1 and y <= 0

def is_true_negative(y, y_pred):
    return y_pred <= 0 and y <= 0

def is_false_negative(y, y_pred):
    return y_pred <= 0 and y >= 1

def verify_predictions(y, y_pred):
    """ returns (tp, fp, tn, fn) """

    tp, fp, tn, fn = 0, 0, 0, 0
    for i, pred in enumerate(y_pred):
        tp += 1 if is_true_positive(y[i], pred) else 0
        fp += 1 if is_false_positive(y[i], pred) else 0
        tn += 1 if is_true_negative(y[i], pred) else 0
        fn += 1 if is_false_negative(y[i], pred) else 0
    return (tp, fp, tn, fn)

def accuracy(y, y_pred):
    tp, fp, tn, fn = verify_predictions(y, y_pred)
    return (tp + tn) / (tp + fp + tn + fn)

def precision(y, y_pred):
    tp, fp, tn, fn = verify_predictions(y, y_pred)
    return tp / (tp + fp)

def recall(y, y_pred):
    tp, fp, tn, fn = verify_predictions(y, y_pred)
    return tp / (tp + fn)

def f1_score(y, y_pred):
    precision_ = precision(y, y_pred)
    recall_ = recall(y, y_pred)
    return 2 * (precision_ * recall_) / (precision_ + recall_)


In [113]:
accuracy_arr, precision_arr, recall_arr, f1_score_arr = [], [], [], []

for i, fold in enumerate(folds):
    test_set = fold
    train_sets = [x for j, x in enumerate(folds) if j != i]
    train_sets = np.vstack(train_sets)

    y_test, y_pred = log_reg_GD(train_sets, test_set, 0.01, 100)

    accuracy_arr.append(accuracy(y_test, y_pred))
    # accuracy_arr.append(metrics.accuracy_score(y_test, y_pred))
    precision_arr.append(precision(y_test, y_pred))
    recall_arr.append(recall(y_test, y_pred))
    f1_score_arr.append(f1_score(y_test, y_pred))

print("10-fold cross validation com regressão logistica (GD)")
print("acurácia: %.8f +/- %.8f" %
      (np.mean(accuracy_arr), np.std(accuracy_arr)))
print("revocação: %.8f +/- %.8f" %
      (np.mean(precision_arr), np.std(precision_arr)))
print("precisão: %.8f +/- %.8f" % (np.mean(recall_arr), np.std(recall_arr)))
print("f1-score: %.8f +/- %.8f" %
      (np.mean(f1_score_arr), np.std(f1_score_arr)))


10-fold cross validation com regressão logistica (GD)
acurácia: 0.94013158 +/- 0.04685034
revocação: 0.95833098 +/- 0.04538283
precisão: 0.95075310 +/- 0.04489893
f1-score: 0.95356293 +/- 0.03454181
